In [ ]:
pip install findspark

In [ ]:
pip install pyspark

In [ ]:
import findspark
findspark.init()

import pandas as pd
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [ ]:
spark = SparkSession.builder\
        .master("local[*]")\
        .appName("PySpark_Clase")\
        .getOrCreate()

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
#1. Realice la carga de la base de datos que va a trabajar en su proyecto final con un esquema de datos propuesto por usted
df = spark.read.csv('/content/Incidentes_2019_tallerPyspark.csv', sep=';', header = True, inferSchema = True )
df.show()

+----+---------------+---------------+---------------------+-------------+----------+----------------+-------------------+------------------+---+------------+---------+--------------------+----------------+--------------------+---------+----------+
| ANO|CLASE_ACCIDENTE|      DIRECCION|DIRECCION ENCASILLADA|       DISENO|EXPEDIENTE| FECHA_ACCIDENTE|   FECHA_ACCIDENTES|GRAVEDAD_ACCIDENTE|MES|NRO_RADICADO|NUMCOMUNA|              BARRIO|          COMUNA|            LOCATION|        X|         Y|
+----+---------------+---------------+---------------------+-------------+----------+----------------+-------------------+------------------+---+------------+---------+--------------------+----------------+--------------------+---------+----------+
|2019| Caida Ocupante|    CR 63 CL 67| CR  063   067  00...| Tramo de via|A000923126|  9/01/2019 4:55|2019-01-09 09:55:00|       Con heridos|  1|     1656008|        5|              Caribe|        Castilla|[-75.5742390745, ...| 834354.7|1184817.25|
|201

In [ ]:
#2. Realice 3 cambios de nombres en columnas
df = df.withColumnRenamed("ANO", "AÑO").withColumnRenamed("DISENO", "DISEÑO").withColumnRenamed("LOCATION","UBICACION")
df.show()

+----+---------------+---------------+---------------------+-------------+----------+----------------+-------------------+------------------+---+------------+---------+--------------------+----------------+--------------------+---------+----------+
| AÑO|CLASE_ACCIDENTE|      DIRECCION|DIRECCION ENCASILLADA|       DISEÑO|EXPEDIENTE| FECHA_ACCIDENTE|   FECHA_ACCIDENTES|GRAVEDAD_ACCIDENTE|MES|NRO_RADICADO|NUMCOMUNA|              BARRIO|          COMUNA|           UBICACION|        X|         Y|
+----+---------------+---------------+---------------------+-------------+----------+----------------+-------------------+------------------+---+------------+---------+--------------------+----------------+--------------------+---------+----------+
|2019| Caida Ocupante|    CR 63 CL 67| CR  063   067  00...| Tramo de via|A000923126|  9/01/2019 4:55|2019-01-09 09:55:00|       Con heridos|  1|     1656008|        5|              Caribe|        Castilla|[-75.5742390745, ...| 834354.7|1184817.25|
|201

In [ ]:
df.dtypes

[('AÑO', 'int'),
 ('CLASE_ACCIDENTE', 'string'),
 ('DIRECCION', 'string'),
 ('DIRECCION ENCASILLADA', 'string'),
 ('DISEÑO', 'string'),
 ('EXPEDIENTE', 'string'),
 ('FECHA_ACCIDENTE', 'string'),
 ('FECHA_ACCIDENTES', 'timestamp'),
 ('GRAVEDAD_ACCIDENTE', 'string'),
 ('MES', 'int'),
 ('NRO_RADICADO', 'int'),
 ('NUMCOMUNA', 'string'),
 ('BARRIO', 'string'),
 ('COMUNA', 'string'),
 ('UBICACION', 'string'),
 ('X', 'double'),
 ('Y', 'double')]

In [ ]:
#3. Realice las respectivas modificaciones de las columnas categóricas a numéricas
dataset = df.select(col('AÑO'),
                    col('CLASE_ACCIDENTE'),
                    col('GRAVEDAD_ACCIDENTE'),
                    col('MES'),
                    col('NRO_RADICADO').cast('string'),
                    col('NUMCOMUNA').cast('int'),
                    col('BARRIO') 
                    )
dataset.dtypes

[('AÑO', 'int'),
 ('CLASE_ACCIDENTE', 'string'),
 ('GRAVEDAD_ACCIDENTE', 'string'),
 ('MES', 'int'),
 ('NRO_RADICADO', 'string'),
 ('NUMCOMUNA', 'int'),
 ('BARRIO', 'string')]

In [ ]:
#4. Realice el borrado de datos faltantes
#4.1Buscar los nulos
dataset.select([count(when(isnull(c), c)).alias(c) for c in dataset.columns]).show()

+---+---------------+------------------+---+------------+---------+------+
|AÑO|CLASE_ACCIDENTE|GRAVEDAD_ACCIDENTE|MES|NRO_RADICADO|NUMCOMUNA|BARRIO|
+---+---------------+------------------+---+------------+---------+------+
|  0|              0|                 0|  0|           0|     2044|  2044|
+---+---------------+------------------+---+------------+---------+------+



In [ ]:
#4.2 Drop missing values
dataset = dataset.replace('null', None)\
        .dropna(how='any')

In [ ]:
#5. Realice 2 queries a su gusto  con el set de datos
df.createOrReplaceTempView('temp_table')

In [ ]:
spark.sql('select * from temp_table where NUMCOMUNA = 2').show()

+----+---------------+------------------+---------------------+-------------+----------+----------------+-------------------+------------------+---+------------+---------+-----------------+----------+--------------------+---------+----------+
| AÑO|CLASE_ACCIDENTE|         DIRECCION|DIRECCION ENCASILLADA|       DISEÑO|EXPEDIENTE| FECHA_ACCIDENTE|   FECHA_ACCIDENTES|GRAVEDAD_ACCIDENTE|MES|NRO_RADICADO|NUMCOMUNA|           BARRIO|    COMUNA|           UBICACION|        X|         Y|
+----+---------------+------------------+---------------------+-------------+----------+----------------+-------------------+------------------+---+------------+---------+-----------------+----------+--------------------+---------+----------+
|2019| Caida Ocupante|      CR 52 CL 109| CR  052   109  00...|Lote o Predio|A000923508|12/01/2019 18:00|2019-01-12 23:00:00|       Con heridos|  1|     1656314|        2|       La Francia|Santa Cruz|[-75.5569834638, ...|836275.88|1188724.05|
|2019|      Atropello|    CL

In [ ]:
spark.sql('select * from temp_table where MES < 6').show()

+----+---------------+---------------+---------------------+-------------+----------+----------------+-------------------+------------------+---+------------+---------+--------------------+----------------+--------------------+---------+----------+
| AÑO|CLASE_ACCIDENTE|      DIRECCION|DIRECCION ENCASILLADA|       DISEÑO|EXPEDIENTE| FECHA_ACCIDENTE|   FECHA_ACCIDENTES|GRAVEDAD_ACCIDENTE|MES|NRO_RADICADO|NUMCOMUNA|              BARRIO|          COMUNA|           UBICACION|        X|         Y|
+----+---------------+---------------+---------------------+-------------+----------+----------------+-------------------+------------------+---+------------+---------+--------------------+----------------+--------------------+---------+----------+
|2019| Caida Ocupante|    CR 63 CL 67| CR  063   067  00...| Tramo de via|A000923126|  9/01/2019 4:55|2019-01-09 09:55:00|       Con heridos|  1|     1656008|        5|              Caribe|        Castilla|[-75.5742390745, ...| 834354.7|1184817.25|
|201